# Платёжеспособность клиентов банка

## Описание проекта
Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить четыре гипотезы:
1. На возврат кредита в срок влияет количество детей клиента.
2. На возврат кредита в срок влияет семейное положение клиента.
3. На возврат кредита в срок влияет уровень дохода клиента. 
4. На возврат кредита в срок влияет цель кредита.

**Ход исследования**
1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез.
4. Подготовка общего вывода.

## Обзор данных

In [1]:
import pandas as pd
df = pd.read_csv('data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице 12 столбцов, 5 текстовых, 5 десятичных, 2 целочисленных.

Согласно документации к данным:
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

В названиях колонок нет нарушений. Количество объектов в колонках `days_employed` и `total_income` (19351) отличается от количества объектов в остальных колонках (21525). Значит, имеются пропуски данных.

**Выводы**

В каждой строке таблицы — данные о клиенте банка. Колонки описывают три вида характеристик клиента: биологические (возраст и пол), социальные (информация о семье и образовании) и финансовые (трудовой стаж, тип занятости, задолженности, доход, цель получения кредита).

Предварительно можно утверждать, что данных достаточно для проверки гипотез. В данных встречаются пропуски, явные и неявные дубликаты, различные артефакты.

Чтобы двигаться дальше, нужно устранить проблемы в данных.


## Предобработка данных

**Обработка пропущенных значений**

In [3]:
df.isna().sum() # подсчитаем суммарное количество пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Всего имеем 2174 пропущенных значения в двух колонках.

In [4]:
len(df[df['days_employed'].isna()]) / len(df['days_employed']) * 100 # подсчитаем долю пропущенных значений в процентах

10.099883855981417

Доля пропущенных значений составила 10%.

In [5]:
df[df['days_employed'].isna()].head(10) # посмотрим первые десять строк с пропусками

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


В представленном отрезке строки с пропусками в колонках `days_employed` и `total_income` совпадают. Учитывая, что количество пропусков в колонках тоже совпадает, можно предположить, что пропуски возникли из-за технической ошибки. По-хорошему, необходимо обратиться к источнику данных, чтобы восполнить пропуски. Однако в учебном проекте такой возможности нет, поэтому заполним пропуски медианными значениями. Медианные значения лучше подходят для количественных данных, т.к. не чувствительны к "выбросам" в данных, когда имеется небольшое количество очень высоких (низких) значений, значительно искажающих среднюю оценку. Данные о ежемесячном доходе как раз имеют выбросы, потому что доля людей получающих очень большие доходы, как правило, мала.   

In [6]:
days_employed_median = df['days_employed'].median() # подсчитаем медианное значение days_employed
total_income_median = df['total_income'].median() # подсчитаем медианное значение total_income

display(days_employed_median)
display(total_income_median)

-1203.369528770489

145017.93753253992

In [7]:
df['days_employed'] = df['days_employed'].fillna(days_employed_median) # заменим данные в колонке days_employed медианным значением
df['total_income'] = df['total_income'].fillna(total_income_median) # заменим данные в колонке total_income медианным значением

df.isna().sum() # подсчитаем суммарное количество пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропущенные значения замененеы на медианные по каждой колонке

**Обработка артефактов** 

Колонка с трудовым стажем имеет отрицательное количество дней, чего не может быть. Видимо, на каком-то этапе сбора данных, к некоторым значениям добавился минус. Также, в этой колонке есть чрезвычайно высокие значения, например 340 266 дней, что составляет 945 лет. Такие высокие значения вызывают вопросы к данным, поэтому их необходимо перепроверять. Но в нашем исследовании трудовой стаж не влияет на проверку гипотез, так что достаточно было заполнить пропуски, чтобы в дальнейшем не возникали ошибки.

Проверим наличие артефактов в количественных колонках, важных для исследования: `children`, `debt`, `total_income`.

In [8]:
df['children'].sort_values().unique() # найдем и отсортируем уникальные значения для колонки с количеством детей


array([-1,  0,  1,  2,  3,  4,  5, 20], dtype=int64)

In [9]:
display(df[df['children'] == -1]['children'].count()) # подсчитаем количество значений -1
display(df[df['children'] == 20]['children'].count()) # подсчитаем количество значений 20

47

76

Имеются значения -1 и 20, которые, вероятно, неверные. Значение -1 могло возникнуть вследствие случайного добавления знака минус, как в случае с колонкой с трудовым стажем, либо этим значением помечали отсутствие данных о количестве детей. Значение 20 детей возможно в реальной жизни, но маловероятно, что в достаточно небольшую выборку попадет такое количество супер-многодетных людей. Скорее всего здесь также ошибка, только теперь случайно был добавлен ноль, а реальное значение два. Необходимо оценить, какую долю будут составлять артефакты, если вернуть их в значения 1 и 2

In [10]:
display(df[df['children'] == -1]['children'].count() / df[df['children'] == 1]['children'].count() * 100) # вычисляем долю -1 в 1, в процентах
display(df[df['children'] == 20]['children'].count() / df[df['children'] == 2]['children'].count() * 100) # вычисляем долю 20 в 2, в процентах

0.9755085097550851

3.6982968369829687

Значения -1 составляли бы около одного процента, значения 20 составляли бы около четырех процентов, при сравнении данных это может вносить небольшую погрешность. Необходимо заменить значения на 1 и 2 соответственно. 

In [11]:
df['children'] = df['children'].replace(-1, 1) # заменяем -1 на 1
df['children'] = df['children'].replace(20, 2) # заменяем 20 на 2

df['children'].sort_values().unique() # проверяем уникальные значения

array([0, 1, 2, 3, 4, 5], dtype=int64)

In [12]:
df['debt'].sort_values().unique() # найдем уникальные значения для колонки с задолженностью

array([0, 1], dtype=int64)

Артефактов нет

In [13]:
display(df['total_income'].max()) # найдем макс. значение для колонки с доходом
display(df['total_income'].min()) # найдем мин. значение для колонки с доходом

2265604.028722744

20667.26379327158

Просмотреть полный перечень уникальных значений дохода затруднительно, поэтому рассмотрены крайние значения выборки. По ним видно, что отрицательных или чрезвычайно завышенных значений в выборке нет, поэтому сделаем вывод что артефактов нет.

**Изменение типа данных**

Цифры после запятой в данных о доходе для нашего исследования не важны, поэтому избавимся от них, изменив тип данных на целочисленный.

In [14]:
df['total_income'] = df['total_income'].astype('int') # изменяем тип данных на целочисленный

df['total_income'].head() # проверяем

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int32

**Обработка дубликатов**

Проверим данные на наличие явных дубликатов. Для этого подойдет метод duplicated(), который проверяет каждое значение, и при наличии дубликата выводит значение True.

In [15]:
df.duplicated().sum() # подсчитаем количество дубликатов

54

In [16]:
df[df.duplicated()].head() # выведем первые пять значений  дубликатов

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,-1203.369529,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,-1203.369529,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,-1203.369529,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,-1203.369529,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,-1203.369529,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы


In [17]:
df = df.drop_duplicates().reset_index(drop=True) # удаляем дубликаты со сбросом индексов
df.duplicated().sum() # подсчитаем количество дубликатов

0

Проверим колонки `education`, `family_status`, `income_type` на наличие неявных дубликатов. Для этого подойдет метод unique(), который выводит список всех уникальных значений в колонке, что позволяет увидеть неявные дубликаты.

In [18]:
df['education'].sort_values().unique() # найдем и отсортируем уникальные значения для колонки с образованием

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Имеются одинаковые значения, написанные в разном регистре. Данные ошибки возникают при ручном вводе значений, особенно если ввод делают разные люди. Необходимо привести значения к нижнему регистру, чтобы избежать повторений.

In [19]:
df['education'] = df['education'].str.lower() # приведем значения к нижнему регистру
df['education'].sort_values().unique() # заново найдем уникальные значения

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

In [20]:
df['family_status'].sort_values().unique()  # найдем и отсортируем уникальные значения для колонки с семейным положением

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

На первый взгляд дубликатов нет, однако непонятно, важны ли для проверки гипотез промежуточные значения 'в разводе', 'вдовец / вдова', 'гражданский брак'. Также, нет ясности в том, что понимается под обозначением 'гражданский брак'. Это может быть обывательское понятие сожительства, либо трактовка понятия 'женат / замужем'. Чтобы это уточнить, необходимо обратиться к источнику данных. Будем считать, что все категории самостоятельны. 

In [21]:
df['income_type'].sort_values().unique()  # найдем и отсортируем уникальные значения для колонки с типом занятости

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

Дубликатов нет.

**Создание новых датафреймов**

Колонки education и family_status имеют дополнительные столбцы с идентификаторами (education_id и family_status_id), поэтому можно сохранить их как отдельные датафреймы, оставив в исходном датафрейме только столбцы с идентификаторами.

In [22]:
df_education = df[['education_id', 'education']] # создаем датафрейм с идентификаторами для образования
df_family_status = df[['family_status_id', 'family_status']] # создаем датафрейм с идентификаторами для семейного положения
df_education = df_education.drop_duplicates().reset_index(drop=True) # удаляем дубликаты и сбрасываем индекс
df_family_status = df_family_status.drop_duplicates().reset_index(drop=True) # удаляем дубликаты и сбрасываем индекс

display(df_education.sort_values(by='education_id')) # отсортировываем и выводим полученный словарь
display(df_family_status.sort_values(by='family_status_id')) # отсортировываем и выводим полученный словарь

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


In [23]:
df = df[[
    'children',
    'days_employed', 
    'dob_years', 
    'education_id', 
    'family_status_id', 
    'gender', 
    'income_type', 
    'debt', 
    'total_income', 
    'purpose'
]] # удаляем из исходного датафрейма education и family_status
df.head() # проверяем датафрейм

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


**Разбивка значений по категориям**

Разобьем значения дохода по категориям:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для этого, напишем функцию, которая будет проходить по строкам в колонке и в зависимости от содержимого присваивать одну из категорий новому столбцу `total_income_category`.

In [24]:
def replace_total_income(total_income): # описываем функцию, присваивающую новые категории
    if 0 <= total_income < 30000:
        return 'E'
    if 30001 <= total_income < 50000:
        return 'D'
    if 50001 <= total_income < 200000:
        return 'C'
    if 200001 <= total_income < 1000000:
        return 'B' 
    if total_income >= 1000001:
        return 'A' 
    return 'ошибка'

replace_total_income(200002) # проверка работы функции

'B'

In [25]:
df['total_income_category'] = df['total_income'].apply(replace_total_income) # применяем нашу функцию к столбцу total_income
df['total_income_category'].sort_values().unique() # проверяем, не было ли ошибок при замене

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

* 'операции с автомобилем',
* 'операции с недвижимостью',
* 'проведение свадьбы',
* 'получение образования'.

In [26]:
df['purpose'].sort_values().unique()  # найдем и отсортируем уникальные значения для колонки с целью кредита

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [27]:
def replace_purpose(purpose): # описываем функцию, присваивающую новые категории
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образ' in purpose:
        return 'получение образования'
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'строит' in purpose:
        return 'операции с недвижимостью'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    return 'ошибка'

replace_purpose('покупка жилья для семьи') # проверка работы функции

'операции с недвижимостью'

In [28]:
df['purpose_category'] = df['purpose'].apply(replace_purpose) # применяем нашу функцию к столбцу purpose
df['purpose_category'].sort_values().unique() # проверяем, не было ли ошибок при замене

array(['операции с автомобилем', 'операции с недвижимостью',
       'получение образования', 'проведение свадьбы'], dtype=object)

## Проверка гипотез

### На возврат кредита в срок влияет количество детей клиента

Проверим гипотезу следующим образом:
* сгруппируем клиентов по количеству детей
* подсчитаем количество должников в каждой группе
* вычислим относительное число должников в каждой группе 

In [29]:
children_grouped_count = df.groupby('children')['children'].count() # группируем клиентов по количеству детей
children_grouped_debt_sum = df.groupby('children')['debt'].sum() # считаем количество должников в каждой группе
children_grouped_ratio = children_grouped_debt_sum / children_grouped_count # вычисляем относительное число должников в каждой группе

children_grouped_ratio

children
0    0.075353
1    0.091639
2    0.094925
3    0.081818
4    0.097561
5    0.000000
dtype: float64

**Выводы**

Полученный результат показывает, что относительное число должников с детьми выше числа должников без детей на ~ 2%. Можно утверждать, что первая гипотеза подтвердилась - должники без детей лучше возвращают кредиты. 

### На возврат кредита в срок влияет семейное положение клиента

Проверим гипотезу следующим образом:
* сгруппируем клиентов по семейному положению
* подсчитаем количество должников в каждой группе
* вычислим относительное число должников в каждой группе 

In [30]:
family_status_id_count = df.groupby('family_status_id')['family_status_id'].count() # группируем клиентов по семейному положению
family_status_id_grouped_debt_sum = df.groupby('family_status_id')['debt'].sum() # считаем количество должников в каждой группе
family_status_id_grouped_ratio = family_status_id_grouped_debt_sum / family_status_id_count # вычисляем относительное число должников в каждой группе

display(df_family_status)
display(family_status_id_grouped_ratio)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


family_status_id
0    0.075421
1    0.093202
2    0.065693
3    0.071130
4    0.097509
dtype: float64

**Выводы**

Полученный результат показывает, что меньше всего должников среди вдовцов, женатых и разведеных клиентов. Больше должников среди неженатых и проживающих в гражданском браке. Можно утверждать, что вторая гипотеза подтвердилась. 

### На возврат кредита в срок влияет уровень дохода клиента

Проверим гипотезу следующим образом:
* сгруппируем клиентов по уровню дохода
* подсчитаем количество должников в каждой группе
* вычислим относительное число должников в каждой группе 

In [31]:
total_income_category_count = df.groupby('total_income_category')['total_income_category'].count() # группируем клиентов по уровню доходов
total_income_category_grouped_debt_sum = df.groupby('total_income_category')['debt'].sum() # считаем количество должников в каждой группе
total_income_category_grouped_ratio = total_income_category_grouped_debt_sum / total_income_category_count # вычисляем относительное число должников в каждой группе

total_income_category_grouped_ratio

total_income_category
A    0.080000
B    0.070621
C    0.084825
D    0.060000
E    0.090909
dtype: float64

**Выводы**

Полученный результат показывает, что худшие заемщики - люди с самым низким уровнем дохода. Меньше всего должников среди людей со средним уровнем дохода. Можно утверждать, что третья гипотеза подтвердилась. 

### На возврат кредита в срок влияет цель кредита

Проверим гипотезу следующим образом:
* сгруппируем клиентов по цели кредита и подсчитаем количество должников в каждой группе с помощью сводной таблицы
* вычислим относительное число должников в каждой группе 

In [35]:
purpose_pivot = df.pivot_table(index='purpose_category', values='debt', aggfunc=['sum', 'count']) # строим сводную таблицу, группируем по цели кредита, в качестве значений выбираем наличие задолженности, применяем функции суммировать и подсчитать количество
purpose_pivot['ratio'] = purpose_pivot['sum'] / purpose_pivot['count'] # вычисляем относительное число должников в каждой группе
purpose_pivot

,sum,count,ratio
,debt,debt,
purpose_category,,,
операции с автомобилем,403,4308,0.093547
операции с недвижимостью,782,10814,0.072314
получение образования,370,4014,0.092177
проведение свадьбы,186,2335,0.079657


**Выводы**

Полученный результат показывает, что хуже всего отдают кредиты на авто и образование. Меньше всего должников среди клиентов, оформивших кредит на недвижимость. Можно утверждать, что четвертая гипотеза подтвердилась.

## Итоги исследования

Мы проверили четыре гипотезы и установили:

1. На возврат кредита в срок влияет наличие детей у клиента - должники без детей лучше возвращают кредиты.

Первая гипотеза подтвердилась.

2. На возврат кредита в срок влияет семейное положение клиента - больше всего должников среди неженатых и проживающих в гражданском браке.

Вторая гипотеза подтвердилась.

3. На возврат кредита в срок влияет уровень дохода клиента - люди с низким уровнем дохода хуже отдают долги.

Третья гипотеза подтвердилась.

4. На возврат кредита в срок влияет цель кредита - меньше всего должников среди клиентов, оформивших кредит на операции с недвижимостью.

Четвертая гипотеза подтвердилась.

